In [ ]:
"""
Enables HiDPI rendering of matplotlib charts on Apple Retina displays.
Comment out the line below if the matplotlib plots don’t look right on your computer.
"""
%config InlineBackend.figure_format = "retina"

# Library imports
import os
from scipy.special import binom
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import numpy as np
from sklearn.preprocessing import normalize

from generator import Zernike_generator
from mip import Microscopy_image_processor
from plot import Plot
from dataset import Dataset

# Keep set to 1 for reproducability of the results.
np.random.seed(1)

In [ ]:
# Download and extract the dataset if it’s not present.

local_data_folder = os.path.join(os.path.abspath(os.getcwd()), "data")
dataset_ulr = "https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz"

Dataset.download_if_missing(local_data_folder, dataset_ulr)

In [ ]:
# Calculate the pupil radius

# Numerical aperture 
na = 1.2

# Pixel size of the microscope’s camera divided by the magnification of the objective’s lens 
pixel_size = 100 # nanometers

# Wavelength of the light used for imaging
wavelength = 250 # nanometers

fractional_pupil_radius = (na * pixel_size) / wavelength

print(f"Fractional pupil radius: {fractional_pupil_radius}")

# Create an instance of the generator

generator = Zernike_generator(64, 3, fractional_pupil_radius)

# Apply defocus
generator.add_aberration(2, 0, 1)

# Apply vertical astigmatism
generator.add_aberration(2, 2, 1)

# Apply horizontal tilt
generator.add_aberration(1, 1, 1)

# Apply oblique astigmatism
generator.add_aberration(2, -2, 1)

Plot.matrix(generator.get_raw_matrix())

In [ ]:
# Get point spread function

Plot.matrix(generator.get_normalized_psf())

In [ ]:
# Load an image

image_file_path = "./data/blotchy_0109.jpg"

image = Microscopy_image_processor.load_as_preprocessed_image(image_file_path, 256)
Plot.normalized_mono_image(image)

In [ ]:
# Change brightness and contrast

image = Microscopy_image_processor.apply_microscopy_visual_atrifacts(image)
Plot.normalized_mono_image(image)

In [ ]:
# Plot a histogram 

Plot.normalized_image_histogram(image)

In [ ]:
# Apply PSF to the image

image = generator.apply_psf_to_image(image)
image = Microscopy_image_processor.inflate_channels(image)
Plot.normalized_mono_image(image)